<a href="https://colab.research.google.com/github/susmitmishra125/QA_NLP/blob/main/NLP_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Importing modules
from nltk.tokenize import word_tokenize, sent_tokenize
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
import torch
from collections import Counter
from tqdm import tqdm
import re
import contractions
from nltk.corpus import stopwords
import string
from collections import defaultdict
import nltk
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')  # For tokenizers
nltk.download('stopwords')


# importing dataset
data = open('passage.txt', 'r', encoding="utf-8").read()
questions = open('question_list.txt', 'r').readlines()

# preprocess for modern method


def preprocessing(rawReadCorpus, complete_preprocess=False):
    pattern = "^a-zA-Z0-9_"
    rawReadCorpus = contractions.fix(rawReadCorpus)
    text_sent = sent_tokenize(rawReadCorpus)  # to split the sentences
    text_sent = [sent.lower() for sent in text_sent]  # to convert to lowercase
    text_sent = ["".join([char for char in text if char not in string.punctuation])
                 for text in text_sent]  # removed punctuation
    text_sent = [word_tokenize(sent) for sent in text_sent]
    # text_sent = ["".join([char for char in text if char not in ]) for text in text_sent] # removed punctuation
    for i in range(len(text_sent)):
        sent = " ".join(text_sent[i])
        sent = re.sub(pattern, ' ', sent)
        sent = sent.replace("“", " ")
        sent = sent.replace("”", " ")
        sent = sent.replace("—", " ")
        sent = sent.replace("_", " ")
        text_sent[i] = sent.split(' ')
        text_sent[i] = [i for i in text_sent[i] if i != '']
        # print(text_sent[i])
    if complete_preprocess:
        ps = nltk.porter.PorterStemmer()
        for i in range(len(text_sent)):
            text_sent[i] = [ps.stem(j) for j in text_sent[i]]
    i = len(text_sent)-1
    while(i >= 0):
        if 'chapter' in text_sent[i]:
            # print(text_sent[i])
            del(text_sent[i])
        i -= 1
    return text_sent


def answer_question(question, answer_text, model, tokenizer):
    '''
    Takes a question string and an answer_text string (which contains the
    answer), and identifies the words within the answer_text that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)
    # Report how long the input sequence is.
    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)
    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1
    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]),  # The tokens representing our input text.
                    # The segment IDs to differentiate question from answer_text
                    token_type_ids=torch.tensor([segment_ids]),
                    return_dict=True)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Start with the first token.
    answer = tokens[answer_start]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    # print('Answer: "' + answer + '"')
    return (answer, start_scores[0][answer_start], end_scores[0][answer_end])


def query(question, text_sent, model, tokenizer, n_sent=3):
    outputs = []
    sent_count = len(text_sent)
    ans = 0
    ans_text = 'NA'
    for i in tqdm(range(sent_count-n_sent+1)):
        segment = text_sent[i:i+n_sent]
        passage = ''
        for sent in segment:
            passage += ' '.join(sent)+'. '
        output = answer_question(question, passage, model, tokenizer)
        outputs.append(
            (output[0], 2*(output[1]*output[2])/(output[1]+output[2])))
        if 2*(output[1]*output[2])/(output[1]+output[2]) >= ans:
            ans = 2*(output[1]*output[2])/(output[1]+output[2])
            ans_text = output[0]
    outputs.sort(key=lambda x: x[1], reverse=True)
    # print('Top 10',outputs[:10])
    cnt = Counter()
    for output in outputs[:10]:
        cnt[output[0]] += 1
    return cnt.most_common()[0][0]


def modern():
    text_sent = preprocessing(data)
    model = BertForQuestionAnswering.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    tokenizer = BertTokenizer.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    for i in range(len(questions)):
        # question in questions:
        questions[i] = questions[i].replace('\n', '')
        print(questions[i])
        ans = query(questions[i], text_sent=text_sent,model = model,tokenizer = tokenizer)
        print(ans)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/mtech1/19CS60R28/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mtech1/19CS60R28/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from numpy import dot
from numpy.linalg import norm

# cos_sim = dot(a, b)/(norm(a)*norm(b))

In [19]:
def traditional():
    print("starting preprocessing")
    text_sent = preprocessing(data,complete_preprocess=False)
    text_sent = [' '.join(sent) for sent in text_sent]
    print("complete preprocessing")
    vectorizer = CountVectorizer(ngram_range=(1, 3))
    
    sentence_vectors = vectorizer.fit_transform(text_sent).toarray()

    for question in questions:
        ans = "NA"
        max_cos = 0
        q_vec = vectorizer.transform([question]).toarray()[0]
        for i in range(len(text_sent)):
            ans_vec = sentence_vectors[i]
            cos_sim = dot(ans_vec,q_vec)/(norm(ans_vec)*norm(q_vec)+1e-9)
            if cos_sim>max_cos:
                # print(sum(q_vec),sum(ans_vec))
                ans = text_sent[i]
                max_cos = cos_sim
        print(question)
        print(ans,max_cos,'\n')
    # print(sentence_vectors.toarr)
traditional()

starting preprocessing
complete preprocessing
what is the colour of eyes of the white rabbit?

down the rabbithole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her 0.35247602884490814 

What is the label on the jar in the shelf?

the duchess 0.3779644729616082 

What was on the solid glass three legged table?

the little door was shut again and the little golden key was lying on the glass table as before and things are worse than ever thought the poor child for i never

# This part is for CoQA with BERT portion

In [ ]:
!git clone https://github.com/NTU-SQUAD/transformers-coqa.git

Cloning into 'transformers-coqa'...
remote: Enumerating objects: 335, done.
remote: Total 335 (delta 0), reused 0 (delta 0), pack-reused 335
Receiving objects: 100% (335/335), 90.65 KiB | 388.00 KiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
# downloading coqa dataset
%cd transformers-coqa/data
!wget 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json'
!wget 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json'
%cd ..

/content/transformers-coqa/data
--2021-10-28 05:42:47--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49001836 (47M) [application/json]
Saving to: ‘coqa-train-v1.0.json’

coqa-train-v1.0.jso 100%[===================>]  46.73M  8.04MB/s    in 6.6s    

2021-10-28 05:42:54 (7.12 MB/s) - ‘coqa-train-v1.0.json’ saved [49001836/49001836]

--2021-10-28 05:42:54--  http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9090845 (8.7M) [application/json]
Saving to: ‘coqa-dev-v1.0.json’

coqa-dev-v1.0.j

In [ ]:
!pip install transformers spacy tensorboard tensorboardX

     |████████████████████████████████| 2.9 MB 13.5 MB/s 
     |████████████████████████████████| 124 kB 18.3 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python3 run_coqa.py --model_type bert \
               --model_name_or_path bert-base-uncased \
               --do_train \
               --do_eval \
               --data_dir data/ \
               --train_file coqa-train-v1.0.json \
               --predict_file coqa-dev-v1.0.json \
               --learning_rate 3e-5 \
               --num_train_epochs 2 \
               --output_dir bert-output/ \
               --do_lower_case \
               --per_gpu_train_batch_size 8  \
               --max_grad_norm -1 \
               --weight_decay 0.01 \
               --adversarial \
               --threads 8


10/28/2021 05:41:29 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
10/28/2021 05:41:30 - DEBUG - filelock -   Attempting to acquire lock 140282094898064 on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
10/28/2021 05:41:30 - DEBUG - filelock -   Lock 140282094898064 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
10/28/2021 05:41:30 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpoir8rb81
Downloading: 100% 433/433 [00:00<00:00, 346kB/s]
10/28/2021 05:41:30 - INFO - transformers.file_utils -   storing 